Criar uma tabela de ordens já com:  
quantidade de itens, vendedores, lojas, mapeamento de Status e valor total da venda (valor total - desconto)

Mapeamento Status:  
1 THEN 'Pending'  
2 THEN 'Processing'  
3 THEN 'Shipped'  
4 THEN 'Delivered'  
ELSE 'Unknown'  

In [0]:
bronze_path   = 'abfss://uc-ext-azure@externalcursodb.dfs.core.windows.net/bikestore/bronze/'
silver_path   = 'abfss://uc-ext-azure@externalcursodb.dfs.core.windows.net/bikestore/silver/'
gold_path     = 'abfss://uc-ext-azure@externalcursodb.dfs.core.windows.net/bikestore/gold/'
resource_path = 'abfss://uc-ext-azure@externalcursodb.dfs.core.windows.net/bikestore/resource/origem/'
resource_path_volume = '/Volumes/infinitybigdata/azure/bikestore_resource/origem/'

In [0]:
# criar varias tabelas temporárias de forma prática
bronze_map = {
    "tmp_bronze_brands":      f"{bronze_path}/brands/",
    "tmp_bronze_categories":  f"{bronze_path}/categories/",
    "tmp_bronze_customers":   f"{bronze_path}/customers/",
    "tmp_bronze_order_items": f"{bronze_path}/order_items/",
    "tmp_bronze_orders":      f"{bronze_path}/orders/",
    "tmp_bronze_products":    f"{bronze_path}/products/",
    "tmp_bronze_staffs":      f"{bronze_path}/staffs/",
    "tmp_bronze_stocks":      f"{bronze_path}/stocks/",
    "tmp_bronze_stores":      f"{bronze_path}/stores/",
}
for view_name, path in bronze_map.items():
    (spark.read.format('delta')
        .load(path)
        .createOrReplaceTempView(view_name))

In [0]:
%sql
select * from tmp_bronze_order_items

In [0]:
%sql
describe tmp_bronze_order_items

In [0]:
%sql
SELECT
  OI.order_id,
  OI.item_id,
  OI.product_id,
  OI.quantity,
  OI.list_price,
  OI.discount,
  --(OI.list_price * OI.quantity) AS value_test,
  ROUND((OI.list_price * OI.quantity) * (1 - OI.discount),2) AS total_price --cálculo pra mostrar preço total com desconto, -                                                         soma quantidade dos itens comprados e remoção de casas decimais
FROM
  tmp_bronze_order_items OI

In [0]:
%sql
SELECT *
FROM 

In [0]:
%sql
-- CTE para calcular o preço total de cada item
WITH QtdProducts AS(
  SELECT
  OI.order_id,
  OI.item_id,
  OI.product_id,
  OI.quantity,
  OI.list_price,
  OI.discount,
  --(OI.list_price * OI.quantity) AS value_test,
  ROUND((OI.list_price * OI.quantity) * (1 - OI.discount),2) AS total_price --cálculo pra mostrar preço total com desconto, -                                                         soma quantidade dos itens comprados e remoção de casas decimais
FROM
  tmp_bronze_order_items OI
)

SELECT
  ORD.order_id,
  ORD.customer_id,
  --Mapeamento de status da compra, 'pendente', 'processando', 'enviado', 'entregue'
  CASE
    WHEN ORD.order_status = 1 THEN 'PENDING'
    WHEN ORD.order_status = 2 THEN 'PROCESSING'
    WHEN ORD.order_status = 3 THEN 'SHIPPED'
    WHEN ORD.order_status = 4 THEN 'DELIVERED'
    ELSE 'UNKNOWN'
    END AS Status,
  --ORD.order_status,
  ORD.order_date,
  ORD.required_date,
  ORD.shipped_date,
  --ORD.store_id,
  --ST.store_id AS store_id2,
  ST.store_name,
  ST.city,
  ST.state,
  --ORD.staff_id
  --STF.staff_id,
  STF.first_name AS first_name_staff,
  STF.email AS staff_email,
  STF.active AS staff_status,
  OIT.product_id,
  OIT.item_id,
  OIT.quantity,
  OIT.list_price,
  OIT.discount,
  OIT.total_price
FROM
  tmp_bronze_orders ORD
LEFT JOIN tmp_bronze_stores ST  ON ORD.store_id = ST.store_id
LEFT JOIN tmp_bronze_staffs STF ON ORD.staff_id = STF.staff_id
LEFT JOIN QtdProducts OIT       ON ORD.order_id = OIT.order_id

In [0]:
df_orders_silver = spark.sql(
    """
  -- CTE para calcular o preço total de cada item
WITH QtdProducts AS(
  SELECT
  OI.order_id,
  OI.item_id,
  OI.product_id,
  OI.quantity,
  OI.list_price,
  OI.discount,
  --(OI.list_price * OI.quantity) AS value_test,
  ROUND((OI.list_price * OI.quantity) * (1 - OI.discount),2) AS total_price --cálculo pra mostrar preço total com desconto, -                                                         soma quantidade dos itens comprados e remoção de casas decimais
FROM
  tmp_bronze_order_items OI
)

SELECT
  ORD.order_id,
  ORD.customer_id,
  --Mapeamento de status da compra, 'pendente', 'processando', 'enviado', 'entregue'
  CASE
    WHEN ORD.order_status = 1 THEN 'PENDING'
    WHEN ORD.order_status = 2 THEN 'PROCESSING'
    WHEN ORD.order_status = 3 THEN 'SHIPPED'
    WHEN ORD.order_status = 4 THEN 'DELIVERED'
    ELSE 'UNKNOWN'
    END AS Status,
  --ORD.order_status,
  ORD.order_date,
  ORD.required_date,
  ORD.shipped_date,
  --ORD.store_id,
  --ST.store_id AS store_id2,
  ST.store_name,
  ST.city,
  ST.state,
  --ORD.staff_id
  --STF.staff_id,
  STF.first_name AS first_name_staff,
  STF.email AS staff_email,
  STF.active AS staff_status,
  OIT.product_id,
  OIT.item_id,
  OIT.quantity,
  OIT.list_price,
  OIT.discount,
  OIT.total_price
FROM
  tmp_bronze_orders ORD
LEFT JOIN tmp_bronze_stores ST  ON ORD.store_id = ST.store_id
LEFT JOIN tmp_bronze_staffs STF ON ORD.staff_id = STF.staff_id
LEFT JOIN QtdProducts OIT       ON ORD.order_id = OIT.order_id
"""
)

# salva em delta
df_orders_silver.write\
    .mode("overwrite")\
    .format("delta")\
    .option("mergeSchema", "true")\
    .save(f"{silver_path}/orders")

In [0]:
%sql
CREATE TABLE bikestore.logistics.orders_silver
AS
SELECT *
FROM delta.`abfss://uc-ext-azure@externalcursodb.dfs.core.windows.net/bikestore/silver/orders`;

In [0]:
%sql
SELECT * FROM bikestore.logistics.orders_silver LIMIT 10